In [33]:
#Important functions you will need
def getLabelAndTimeStartAndEnd(filename):
    returnList = []
    with open(filename) as file:
        for line in file:
            if len(line.split()) == 4:
                returnList.append(line.split()[:-1])
    return returnList

def get_info_with_mne_tina(file_path):
    """ read info from the edf file without loading the data. loading data is done in multiprocessing since it takes
    some time. getting info is done before because some files had corrupted headers or weird sampling frequencies
    that caused the multiprocessing workers to crash. therefore get and check e.g. sampling frequency and duration
    beforehand
    :param file_path: path of the recording file
    :return: file name, sampling frequency, number of samples, number of signals, signal names, duration of the rec
    """
    try:
        f = mne.io.read_raw_edf(file_path+".edf", verbose='error')
        labelList = getLabelAndTimeStartAndEnd(file_path+".tse")
    except ValueError:
        return None, None, None, None, None, None

    
    samplingFrequency = int(f.info['sfreq'])
    if samplingFrequency < 10:
        samplingFrequency = 1 / (f.times[1] - f.times[0])
        if samplingFrequency < 10:
            return None, sampling_frequency, None, None

    # edf_file, sampling_frequency, n_samples, signal_names
    # remember that duration = n_samples / sampling_frequency 
    return f, samplingFrequency, f.n_times, f.ch_names, labelList

def get_info_with_mne(file_path):
    """ read info from the edf file without loading the data. loading data is done in multiprocessing since it takes
    some time. getting info is done before because some files had corrupted headers or weird sampling frequencies
    that caused the multiprocessing workers to crash. therefore get and check e.g. sampling frequency and duration
    beforehand
    :param file_path: path of the recording file
    :return: file name, sampling frequency, number of samples, number of signals, signal names, duration of the rec
    """
    try:
        edf_file = mne.io.read_raw_edf(file_path, verbose='error')
    except ValueError:
        return None, None, None, None, None, None
        # fix_header(file_path)
        # try:
        #     edf_file = mne.io.read_raw_edf(file_path, verbose='error')
        #     logging.warning("Fixed it!")
        # except ValueError:
        #     return None, None, None, None, None, None

    # some recordings have a very weird sampling frequency. check twice before skipping the file
    sampling_frequency = int(edf_file.info['sfreq'])
    if sampling_frequency < 10:
        sampling_frequency = 1 / (edf_file.times[1] - edf_file.times[0])
        if sampling_frequency < 10:
            return None, sampling_frequency, None, None, None, None

    n_samples = edf_file.n_times
    signal_names = edf_file.ch_names
    n_signals = len(signal_names)
    # some weird sampling frequencies are at 1 hz or below, which results in division by zero
    duration = n_samples / max(sampling_frequency, 1)

    # TODO: return rec object?
    return edf_file, sampling_frequency, n_samples, n_signals, signal_names, duration





In [44]:
def compute_pyeeg_feats(rec):
    # these values are taken from the tuh paper
    TAU, DE, Kmax = 4, 10, 5
    pwrs, pwrrs, pfds, hfds, mblts, cmplxts, ses, svds, fis, hrsts = [], [], [], [], [], [], [], [], [], []
    dfas, apes = [], []

    for window_id, window in enumerate(rec.signals):
        for window_electrode_id, window_electrode in enumerate(window):
            # taken from pyeeg code / paper
            electrode_diff = list(np.diff(window_electrode))
            M = pyeeg.embed_seq(window_electrode, TAU, DE)
            W = scipy.linalg.svd(M, compute_uv=False)
            W /= sum(W)

            power, power_ratio = self.bin_power(window_electrode, self.bands, rec.sampling_freq)
            pwrs.extend(list(power))
            # mean of power ratio is 1/(len(self.bands)-1)
            pwrrs.extend(list(power_ratio))

            pfd = pyeeg.pfd(window_electrode, electrode_diff)
            pfds.append(pfd)

            hfd = pyeeg.hfd(window_electrode, Kmax=Kmax)
            hfds.append(hfd)

            mobility, complexity = pyeeg.hjorth(window_electrode, electrode_diff)
            mblts.append(mobility)
            cmplxts.append(complexity)

            se = self.spectral_entropy(window_electrode, self.bands, rec.sampling_freq, power_ratio)
            ses.append(se)

            svd = pyeeg.svd_entropy(window_electrode, TAU, DE, W=W)
            svds.append(svd)

            fi = pyeeg.fisher_info(window_electrode, TAU, DE, W=W)
            fis.append(fi)

            # this crashes...
            # ape = pyeeg.ap_entropy(electrode, M=10, R=0.3*np.std(electrode))
            # apes.append(ape)

            # takes very very long to compute
            # hurst = pyeeg.hurst(electrode)
            # hrsts.append(hurst)

            # takes very very long to compute
            # dfa = pyeeg.dfa(electrode)
            # dfas.append(dfa)

    pwrs = np.asarray(pwrs).reshape(rec.signals.shape[0], rec.signals.shape[1], len(self.bands)-1)
    pwrs = np.mean(pwrs, axis=0)

    pwrrs = np.asarray(pwrrs).reshape(rec.signals.shape[0], rec.signals.shape[1], len(self.bands)-1)
    pwrrs = np.mean(pwrrs, axis=0)

    pfds = np.asarray(pfds).reshape(rec.signals.shape[0], rec.signals.shape[1])
    pfds = np.mean(pfds, axis=0)

    hfds = np.asarray(hfds).reshape(rec.signals.shape[0], rec.signals.shape[1])
    hfds = np.mean(hfds, axis=0)

    mblts = np.asarray(mblts).reshape(rec.signals.shape[0], rec.signals.shape[1])
    mblts = np.mean(mblts, axis=0)

    cmplxts = np.asarray(cmplxts).reshape(rec.signals.shape[0], rec.signals.shape[1])
    cmplxts = np.mean(cmplxts, axis=0)

    ses = np.asarray(ses).reshape(rec.signals.shape[0], rec.signals.shape[1])
    ses = np.mean(ses, axis=0)

    svds = np.asarray(svds).reshape(rec.signals.shape[0], rec.signals.shape[1])
    svds = np.mean(svds, axis=0)

    fis = np.asarray(fis).reshape(rec.signals.shape[0], rec.signals.shape[1])
    fis = np.mean(fis, axis=0)

    return list(pwrs.ravel()), list(pwrrs.ravel()), pfds, hfds, mblts, cmplxts, ses, svds, fis, apes, hrsts, dfas

In [35]:
    def generate_features_perrec(self, rec):
        """ computes features. returns one feature vector per recording
        :param rec: rec.signals has shape n_windows x n_electrodes x n_samples_in_window
                    rec.signals_ft has shape n_windows x n_electrodes x n_samples_in_window/2 + 1
        :return: the feature vector of that recording
        """
        # the window size in seconds can be float. make sure that window_size is int. if not pandas rolling crashes
        window_size = int(self.window_size_sec * rec.sampling_freq)
        features = list()

        # adds 2 patient features
        # TODO: add medication, ...?
        if self.patient_feat_flag:
            features.append(rec.sex)  # does not seem to have any influence at all
            features.append(rec.age)  # seems to have high influence

########################################################################################################################
        # frequency features
        # computes n_features * n_bands * n_electrodes features
        # TODO: test
        if self.freq_feat_flag:
            for freq_feat_name in self.freq_feats:
                # func = getattr(features_frequency, freq_feat_name)
                feature_values = self.compute_freq_feats(freq_feat_name, rec)
                features.extend(feature_values)

########################################################################################################################
        # time features speeded up by computation on whole signals with a rolling window
        # TODO: ".diff() in computation moves results by 1 position", how to pick the correct values from the rolling?!
        if self.time_feat_flag:
            for time_feat in self.time_feats:
                func = getattr(features_time, time_feat)
                # TODO: don't cast the signals here, originally safe them as pandas DataFrame
                feature_values = func(pd.DataFrame(rec.signals_complete.T), window_size)
                feature_values = self.rolling_to_windows(feature_values, window_size)
                feature_values = np.mean(feature_values, axis=0)
                features.extend(feature_values)

########################################################################################################################
        # These values are taken from the pyeeg paper
        # pyeeg features. hurst and dfa have a very, very high computation time.
        # implementations are given on 1D time series only. therefore pass every channel of the recording to the module
        # computes 7*n_elec + 2*n_bands*n_elecs values
        #  TODO: test and speed up
        if self.pyeeg_feat_flag:
            pwrs, pwrrs, pfds, hfds, mblts, cmplxts, ses, svds, fis, apes, hrsts, dfas = self.compute_pyeeg_feats(rec)

            features.extend(pwrs)
            features.extend(pwrrs)
            features.extend(pfds)
            features.extend(hfds)
            features.extend(mblts)
            features.extend(cmplxts)
            features.extend(ses)
            features.extend(svds)
            features.extend(fis)

            # features.extend(apes)
            # features.extend(hrsts)
            # features.extend(dfas)

########################################################################################################################
        # correlation features
        # takes too long to compute on whole signals
        # correlate opposite electrodes?
        # TODO: add
        if self.corr_feat_flag:
            autocorrelations, crosscorrelations = [], []

            features.extend(autocorrelations)
            features.append(crosscorrelations)

########################################################################################################################
        # dwt features
        # computes 3*n_elecs*n_lvls + n_elecs*(n_lvls-1)
        # TODO: check sampling frequency and maybe resample to achieve equal sub-bands?
        # TODO: try the "morlet" wavelet -> cwt?
        # TODO: test
        if self.dwt_feat_flag:
            wavelet = pywt.Wavelet('db4')
            level = pywt.dwt_max_level(window_size, wavelet.dec_len)
            if level > 5:
                level = 5
            # TODO: handle this case
            elif level < 5:
                logging.warning("Recording {} is too short to perform a level-{} dwt!".format(rec.name, 5))

            # rec_coeffs is in shape: n_levels x n_electrodes x n_band_coeffs
            rec_coeffs = pywt.wavedec(rec.signals_complete, wavelet=wavelet, level=level)
            # don't use band d1?
            # rec_coeffs = rec_coeffs[1:]

            means, avg_powers, stds, ratios = self.compute_dwt_feats(rec_coeffs)
            features.extend(means)
            features.extend(avg_powers)
            features.extend(stds)
            features.extend(ratios)

        return features

In [36]:
def process(self, rec):
    """ each file is first read, then cleaned, the split with the specified window function and transformed with
    fourier transformation
    """
    rec.signals = my_io.load_data_with_mne(rec)
    rec = self.cleaner.clean(rec)

    # are standard signals needed somewhere? or can they be overwritten?
    rec.signals_complete = rec.signals
    windows = self.splitter.split(rec)
    rec.signals = windows
    rec.signals_ft = np.fft.rfft(windows, axis=2)

    rec.signals_ft = np.abs(rec.signals_ft)
    return rec

In [37]:
class Recording(object):
    """ This is a container class for all the relevant data of a EEG recording
    """

# ______________________________________________________________________________________________________________________
    def __init__(self, data_set, name, raw_edf, sampling_freq, n_samples, n_signals, signal_names, duration,
                 signals=None, signals_complete=None, signals_ft=None, sex=None, age=None):
        self.data_set = data_set
        self.name = name
        self.raw_edf = raw_edf
        self.sampling_freq = sampling_freq
        self.n_samples = n_samples
        self.n_signals = n_signals
        self.signal_names = signal_names
        self.duration = duration
        self.signals = signals
        self.signal_ft = signals_ft
        self.signals_complete = signals_complete
        self.sex = sex
        self.age = age



In [ ]:
def write_hdf5(features, in_dir, cmd_args):
    """ writes features to hdf5 file
    :param features: a matrix holding a feature vector for every recordings (maybe someday for every time window of
    every recording)
    :param in_dir: input directory used to extract the name if the class
    :param cmd_args: used to include important information in the file name s.a. window, windowsize etc
    :return: the name of the feature file
    """
    data_set = in_dir.split('/')[-2]
    file_name = os.path.join(cmd_args.output, data_set,
                             '_'.join([data_set, '-'.join([cmd_args.window,
                                                           str(cmd_args.windowsize)+'s',
                                                           str(cmd_args.overlap)+'%']),
                                       cmd_args.bands.replace(',', '-')])) + '.hdf5'

    logging.info("\t\tWriting features to {}.\n".format(file_name))

    hdf5_file = h5py.File(file_name, 'w')
    hdf5_file.create_dataset('data', features.shape, data=features)
    hdf5_file.close()

    return file_name

In [2]:
class Metadata(object):

    def __init__(self):
        self.shape = None
        self.data_length_sec = None
        self.sampling_frequency = None
        self.channels = None
        self.sequences = []

    def add_shape(self, shape):
        if self.shape is None:
            self.shape = shape
        else:
            assert shape == self.shape

    def add_data_length_sec(self, data_length_sec):
        if self.data_length_sec is None:
            self.data_length_sec = data_length_sec
        else:
            assert data_length_sec == self.data_length_sec

    def add_sampling_frequency(self, sampling_frequency):
        if self.sampling_frequency is None:
            self.sampling_frequency = sampling_frequency
        else:
            assert sampling_frequency == self.sampling_frequency

    def add_channels(self, channels):
        if self.channels is None:
            self.channels = channels
        else:
            assert np.alltrue(channels == self.channels)

    def add_sequence(self, sequence):
        if sequence is not None:
            self.sequences.append(sequence)


In [3]:
class Windower:
    """
    Breaks the time-series data into N second segments, for example 60s windows
    will create 10 windows given a 600s segment. The output is the reshaped data
    e.g. (600, 120000) -> (600, 10, 12000)
    """
    def __init__(self, window_secs=None):
        self.window_secs = window_secs
        self.name = 'w-%ds' % window_secs if window_secs is not None else 'w-whole'

    def get_name(self):
        return self.name

    def apply(self, X, meta=None):
        if self.window_secs is None:
            return X.reshape([1] + list(X.shape))

        num_windows = meta.data_length_sec / self.window_secs
        samples_per_window = self.window_secs * int(meta.sampling_frequency)
        samples_used = num_windows * samples_per_window
        samples_dropped = X.shape[-1] - samples_used
        X = Slice(samples_dropped).apply(X)
        out = np.split(X, num_windows, axis=X.ndim-1)
        out = to_np_array(out)
        return out


In [4]:
def to_np_array(X):
    if isinstance(X[0], np.ndarray):
        # return np.vstack(X)
        out = np.empty([len(X)] + list(X[0].shape), dtype=X[0].dtype)
        for i, x in enumerate(X):
            out[i] = x
        return out

    return np.array(X)

In [46]:
def load_data_with_mne(rec):
    """ loads the data using the mne library
    :param rec: recording object holding all necessary data of an eeg recording
    :return: a pandas dataframe holding the data of all electrodes as specified in the rec object
    """
    rec.raw_edf.load_data()
    signals = rec.raw_edf.get_data()

    data = pd.DataFrame(index=range(rec.n_samples), columns=rec.signal_names)
    for electrode_id, electrode in enumerate(rec.signal_names):
        data[electrode] = signals[electrode_id]

    # TODO: return rec object?
    return data

In [5]:
class Slice:
    """
    Take a slice of the data on the last axis.
    e.g. Slice(1, 48) works like a normal python slice, that is 1-47 will be taken
    """
    def __init__(self, start, end=None):
        self.start = start
        self.end = end

    def get_name(self):
        return "slice%d%s" % (self.start, '-%d' % self.end if self.end is not None else '')

    def apply(self, data, meta=None):
        s = [slice(None),] * data.ndim
        s[-1] = slice(self.start, self.end)
        return data[s]

In [7]:
%matplotlib qt
%matplotlib inline

#remember to do "conda activate mne" before launching the jupyter notebook
from functools import partial
import multiprocessing as mp
import numpy as np
import pandas as pd
import logging
import os
import mne

prePath = "/Users/tinaraissi/workspace/EEG/tuh-eeg-auto-diagnosis/"


In [8]:
def getDataframe(filename):
    #this function read an edf file and returns a dataframe (n_samples * n_channels)
    # having the time samples for each electrode in each column
    
    edfData =  mne.io.read_raw_edf(filename)
    dataFromEdf = edfData.get_data()
    dataset = pd.DataFrame(index=range(edfData.n_times), columns=edfData.ch_names)

    #At this point you have 
    for dataSample, channel in enumerate(edfData.ch_names):
        dataset[channel] = dataFromEdf[dataSample]
        
    return edfData, dataset        

In [13]:
#File formats in the relative folder
#*.edf:    the EEG sampled data in European Data Format (edf)
#*.txt:    the EEG report corresponding to the patient and session
#*.tse:    term-based annotations using all available seizure type classes
#*.tse_bi: same as *.tse except bi-class annotations (seizure/background) 
#*.lbl:    event-based annotations using all available seizure type classes
#*.lbl_bi: same as *.lbl except bi-class annotations (seizure/background)

filename = prePath+"v1.4.0/edf/train/02_tcp_le/001/00000143/s001_2003_03_10/00000143_s001_t001.edf"
#filename = "v1.4.0/edf/train/02_tcp_le/001/00000143/s001_2003_03_10/00000143_s001_t001.edf"
edfData, dataset =  getDataframe(filename)


Extracting EDF parameters from /Users/tinaraissi/workspace/EEG/tuh-eeg-auto-diagnosis/v1.4.0/edf/train/02_tcp_le/001/00000143/s001_2003_03_10/00000143_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [10]:
labelFilename = "v1.4.0/edf/train/02_tcp_le/001/00000143/s001_2003_03_10/00000143_s001_t001.tse"

In [11]:
def init_processing_units():
    self.cleaner = data_cleaner.DataCleaner(elecs=cmd_args.elecs)
    self.splitter = data_splitter.DataSplitter(window=cmd_args.window, window_size_sec=cmd_args.windowsize,
                                               overlap=cmd_args.overlap)
    self.feature_generator = feature_generator.FeatureGenerator(domain=cmd_args.domain, bands=cmd_args.bands,
                                                                window_size_sec=cmd_args.windowsize,
                                                                overlap=cmd_args.overlap, perrec=cmd_args.perrec,
                                                                electrodes=self.cleaner.get_electrodes())

In [14]:
filename.title()

'/Users/Tinaraissi/Workspace/Eeg/Tuh-Eeg-Auto-Diagnosis/V1.4.0/Edf/Train/02_Tcp_Le/001/00000143/S001_2003_03_10/00000143_S001_T001.Edf'

In [17]:
#rootdir = "v1.4.0/edf/train/02_tcp_le"
rootdir = prePath+"v1.4.0/edf/train/02_tcp_le/"
segLabelFilenames = {}


for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        p = os.path.join(subdir, file)
        if p.endswith("edf"):
            segLabelFilenames[p[56:-4]] = p.split(".edf")[0]
        

In [24]:
p = list(segLabelFilenames.keys())[10]

In [25]:
segLabelFilenames[p]
f = get_info_with_mne(segLabelFilenames[p])

In [26]:
getLabelAndTimeStartAndEnd(segLabelFilenames[p]+".tse")

[['0.0000', '531.0000', 'bckg']]

In [17]:
88000/250

352.0

In [27]:
f

(<RawEDF  |  00002445_s002_t001.edf, n_channels x n_times : 41 x 132750 (531.0 sec), ~86 kB, data not loaded>,
 250,
 132750,
 ['EEG FP1-LE',
  'EEG FP2-LE',
  'EEG F3-LE',
  'EEG F4-LE',
  'EEG C3-LE',
  'EEG C4-LE',
  'EEG A1-LE',
  'EEG A2-LE',
  'EEG P3-LE',
  'EEG P4-LE',
  'EEG O1-LE',
  'EEG O2-LE',
  'EEG F7-LE',
  'EEG F8-LE',
  'EEG T3-LE',
  'EEG T4-LE',
  'EEG T5-LE',
  'EEG T6-LE',
  'EEG FZ-LE',
  'EEG CZ-LE',
  'EEG PZ-LE',
  'EEG OZ-LE',
  'EEG PG1-LE',
  'EEG PG2-LE',
  'EEG EKG-LE',
  'EEG 26-LE',
  'EEG 27-LE',
  'EEG 28-LE',
  'EEG 29-LE',
  'EEG 30-LE',
  'EEG 31-LE',
  'EEG 32-LE',
  'PHOTIC PH',
  'DC1-DC',
  'DC2-DC',
  'DC3-DC',
  'DC4-DC',
  'DC5-DC',
  'DC6-DC',
  'DC7-DC',
  'DC8-DC'],
 [['0.0000', '531.0000', 'bckg']])

In [29]:
selected_ch_names = []

wanted_elecs = ['A1', 'A2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1',
                'FP2', 'FZ', 'O1', 'O2',
                'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']

for wanted_part in wanted_elecs:
    wanted_found_name = []
    for ch_name in f[3]:
        if ' ' + wanted_part + '-' in ch_name:
            wanted_found_name.append(ch_name)
    assert len(wanted_found_name) == 1
    selected_ch_names.append(wanted_found_name[0])

In [30]:
selected_ch_names

['EEG A1-LE',
 'EEG A2-LE',
 'EEG C3-LE',
 'EEG C4-LE',
 'EEG CZ-LE',
 'EEG F3-LE',
 'EEG F4-LE',
 'EEG F7-LE',
 'EEG F8-LE',
 'EEG FP1-LE',
 'EEG FP2-LE',
 'EEG FZ-LE',
 'EEG O1-LE',
 'EEG O2-LE',
 'EEG P3-LE',
 'EEG P4-LE',
 'EEG PZ-LE',
 'EEG T3-LE',
 'EEG T4-LE',
 'EEG T5-LE',
 'EEG T6-LE']

In [31]:
len(wanted_elecs)

21

In [39]:
rec = Recording("/", filename, *get_info_with_mne(filename))

In [40]:
rec

In [41]:
vars(rec)

{'data_set': '/',
 'name': '/Users/tinaraissi/workspace/EEG/tuh-eeg-auto-diagnosis/v1.4.0/edf/train/02_tcp_le/001/00000143/s001_2003_03_10/00000143_s001_t001.edf',
 'raw_edf': <RawEDF  |  00000143_s001_t001.edf, n_channels x n_times : 33 x 319750 (1279.0 sec), ~70 kB, data not loaded>,
 'sampling_freq': 250,
 'n_samples': 319750,
 'n_signals': 33,
 'signal_names': ['EEG FP1-LE',
  'EEG FP2-LE',
  'EEG F3-LE',
  'EEG F4-LE',
  'EEG C3-LE',
  'EEG C4-LE',
  'EEG A1-LE',
  'EEG A2-LE',
  'EEG P3-LE',
  'EEG P4-LE',
  'EEG O1-LE',
  'EEG O2-LE',
  'EEG F7-LE',
  'EEG F8-LE',
  'EEG T3-LE',
  'EEG T4-LE',
  'EEG T5-LE',
  'EEG T6-LE',
  'EEG FZ-LE',
  'EEG CZ-LE',
  'EEG PZ-LE',
  'EEG OZ-LE',
  'EEG PG1-LE',
  'EEG PG2-LE',
  'EEG EKG-LE',
  'EEG SP2-LE',
  'EEG SP1-LE',
  'EEG 28-LE',
  'EEG 29-LE',
  'EEG 30-LE',
  'EEG T1-LE',
  'EEG T2-LE',
  'PHOTIC PH'],
 'duration': 1279.0,
 'signals': None,
 'signal_ft': None,
 'signals_complete': None,
 'sex': None,
 'age': None}

In [45]:
compute_pyeeg_feats(rec)

TypeError: 'NoneType' object is not iterable

In [47]:
rec.signals = load_data_with_mne(rec)

In [48]:
compute_pyeeg_feats(rec)

AxisError: axis -1 is out of bounds for array of dimension 0

In [49]:
rec.signals

,EEG FP1-LE,EEG FP2-LE,EEG F3-LE,EEG F4-LE,EEG C3-LE,EEG C4-LE,EEG A1-LE,EEG A2-LE,EEG P3-LE,EEG P4-LE,...,EEG PG2-LE,EEG EKG-LE,EEG SP2-LE,EEG SP1-LE,EEG 28-LE,EEG 29-LE,EEG 30-LE,EEG T1-LE,EEG T2-LE,PHOTIC PH
0,-3.601076e-05,-0.000027,-0.000051,-0.000010,-4.272463e-05,-0.000060,-0.000005,-0.000038,-0.000078,-0.000075,...,0.000253,-0.000006,3.286745e-04,0.000109,-0.000044,-0.000084,0.000070,0.005000,0.005000,0.0
1,-3.356935e-05,-0.000035,-0.000061,-0.000018,-5.554202e-05,-0.000068,-0.000010,-0.000033,-0.000087,-0.000081,...,0.000245,-0.000019,3.222658e-04,0.000042,-0.000049,-0.000095,0.000069,-0.001059,0.003598,0.0
2,-2.868654e-05,-0.000034,-0.000061,-0.000016,-5.798343e-05,-0.000067,-0.000014,-0.000030,-0.000087,-0.000077,...,0.000244,-0.000023,3.228761e-04,0.000005,-0.000049,-0.000091,0.000075,-0.005000,0.002186,0.0
3,-3.570558e-05,-0.000032,-0.000057,-0.000016,-5.249026e-05,-0.000065,-0.000013,-0.000027,-0.000086,-0.000076,...,0.000251,-0.000021,3.305055e-04,0.000073,-0.000046,-0.000082,0.000078,0.001625,0.003958,0.0
4,-3.356935e-05,-0.000031,-0.000052,-0.000012,-5.035403e-05,-0.000063,-0.000011,-0.000033,-0.000080,-0.000078,...,0.000252,-0.000021,3.295900e-04,0.000119,-0.000045,-0.000079,0.000070,0.005000,0.005000,0.0
5,-2.197267e-05,-0.000025,-0.000049,-0.000008,-4.425051e-05,-0.000060,-0.000006,-0.000038,-0.000075,-0.000077,...,0.000245,-0.000017,3.213503e-04,0.000051,-0.000044,-0.000081,0.000069,0.000259,0.003910,0.0
6,-1.434327e-05,-0.000021,-0.000045,-0.000005,-4.364016e-05,-0.000055,-0.000008,-0.000034,-0.000075,-0.000069,...,0.000244,-0.000021,3.222658e-04,0.000014,-0.000045,-0.000078,0.000073,-0.005000,0.002342,0.0
7,-1.678468e-05,-0.000016,-0.000040,-0.000003,-3.662111e-05,-0.000051,-0.000009,-0.000034,-0.000070,-0.000064,...,0.000249,-0.000019,3.274538e-04,0.000052,-0.000041,-0.000068,0.000075,0.000059,0.003388,0.0
8,-2.563478e-05,-0.000020,-0.000041,-0.000005,-3.662111e-05,-0.000052,-0.000009,-0.000035,-0.000069,-0.000069,...,0.000251,-0.000019,3.225710e-04,0.000118,-0.000047,-0.000071,0.000065,0.005000,0.005000,0.0
9,-1.312257e-05,-0.000018,-0.000038,-0.000002,-3.875734e-05,-0.000053,-0.000011,-0.000032,-0.000071,-0.000073,...,0.000246,-0.000024,3.137209e-04,0.000053,-0.000047,-0.000075,0.000059,0.001652,0.004276,0.0
